### 3.1 Simple Robbins Monro

Write the first order budget constraint as :$$F(w) = E[(R-\mu)(R-\mu)']w + \lambda \mathbf{1}_n = Sw + \lambda \mathbf{1}_n=0,$$ 
We can write $F(w) = E(f(w, R)),$ where $$f(w,R) = (R-\mu)(R-\mu)'w + \lambda\mathbf{1}_n$$
We can easily verify the 2 hypothesis: 

* if $w*$ is the solution, $\forall w\in \mathbb{R}^n$,
\begin{eqnarray}
 \langle w - w*, F(w)\rangle &=& \langle w-w*, Sw + \lambda \mathbf{1}_n\rangle \\
 &=& \langle w-w*, S(w-w*) + Sw*+ \lambda \mathbf{1}_n\rangle \\
 &=&\langle w-w*, S(w-w*)\rangle >0
\end{eqnarray}

* $\forall w\in \mathbb{R}^n, E(f^2(w, R)) = E(w^TA^2w+2\lambda w^TA\mathbf{1}_n+n\lambda^2)= w^TE(A^2)w + 2\lambda w^TS\mathbf{1}_n +n\lambda^2\leq C(1+|w|^2),$ when $C$ is quite large. 

In [1]:
import numpy as np
import scipy 
import scipy.linalg
from tqdm import tqdm

In [2]:
n = 3
mu = np.array([.05, .07, .06])
D = np.diag([.1, .14, .02])
rho = 0.5
K = np.array([[1, rho, 0],
             [rho, 1, rho],
             [0, rho, 1]])

S = np.matmul(np.matmul(D, K), D)

In [10]:
def f(w, x):
    t1 = x.reshape(n, 1)
    t2 = mu.reshape(n, 1)
    return np.matmul((t1 - t2)*((t1 - t2).T), w) + 0.0003*np.ones((n, 1))+t1


In [2]:
from tqdm import tqdm

In [8]:
w0 = np.ones((n, 1))
N = 1000000
R = np.random.multivariate_normal(mu, S, N)
for i in tqdm(range(N)):
    w0 = w0 - 1000*f(w0, R[i])/(i+1)
    
w0 = w0/np.sum(w0)
w0    

100%|███████████████████████████████████████████████████████████████████| 10000000/10000000 [01:44<00:00, 96023.75it/s]


array([[ 0.09030035],
       [-0.08923585],
       [ 0.9989355 ]])

In [9]:
np.dot(S,w0)

array([[0.00027835],
       [0.00028159],
       [0.00027464]])

In [17]:
w0 = np.ones((n, 1))
N = 1000000
R = np.random.multivariate_normal(mu, S, N)
for i in tqdm(range(N)):
    w0 = w0 - 100*f(w0, R[i])/(i+1)
    
w0 = w0/np.sum(w0)
w0 

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:11<00:00, 90842.71it/s]


array([[ 0.10348786],
       [-0.04953556],
       [ 0.9460477 ]])

#### True Solution
$$w = \frac{S^{-1}\mathbf{1}_n}{\mathbf{1}_n'S^{-1}\mathbf{1}_n}$$

In [ ]:
np.dot(inv, np.ones(3))

In [6]:
inv = np.linalg.inv(S)
w = np.matmul(inv, np.ones((n, 1)))/np.sum(inv)
w

array([[ 0.09014558],
       [-0.08958567],
       [ 0.99944009]])

## 3.2

In [95]:
def f(w, x):
    t1 = x.reshape(n, 1)
    t2 = mu.reshape(n, 1)
    return np.matmul((t1 - t2)*((t1 - t2).T), w) + 0.0003*np.ones((n, 1))+t1


In [96]:
def S_rho(rho):
    D = np.diag([.1, .14, .02])
    # rho = 0.5
    K = np.array([[1, rho, 0],
                 [rho, 1, rho],
                 [0, rho, 1]])

    S = np.matmul(np.matmul(D, K), D)
    return S

In [109]:
def UQSA( mu, rho_min, rho_max, Mk, u0 = '', K = 10000, mk = 6, n=3,step_multi = 1.0):
    if u0=='' : 
        u0 = np.random.randn(mk,n)
    u = np.zeros(shape = (mk,n))    
    assert len(u0) == mk
    
    Legendre_basis = [np.polynomial.legendre.Legendre.basis(deg=i) for i in range(mk)]
        
    def W(rho,u0,Legendre_basis = Legendre_basis):
        return np.sum([u0[j]*Legendre_basis[j](rho) for j in range(mk)],axis = 0)
    
    for k in tqdm(range(K)):
        
        rho = []
        R = []
        for s in range(Mk):
            r = np.random.uniform(low = rho_min, high = rho_max)
            rho.append(r)
            S = S_rho(r)
            R.append(np.random.multivariate_normal(mu, S))
            
        for i in range(mk):
            expect = 0.0
            for s in range(Mk):
                w = W(rho[s],u0)
                w = w.reshape(n,1)
                expect = expect + f(w,R[s])*Legendre_basis[i](rho[s])
            expect = expect.reshape(1,n) / Mk
            
            u[i] = u0[i] - step_multi * expect/(k+1)
            
        u0 = u
#     Sum = np.sum(u)
    return [x/np.sum(x) for x in u]  #TODO

In [106]:
n = 3
mu = np.array([.05, .07, .06])
mu.reshape(3,1)

array([[0.05],
       [0.07],
       [0.06]])

In [111]:
UQSA(mu, 0.6,0.8,100,K = 10000)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [09:43<00:00, 17.13it/s]


[array([ 0.74497849, -0.08957938,  0.34460089]),
 array([-0.14713907,  2.64592406, -1.49878499]),
 array([0.12922103, 0.22216152, 0.64861745]),
 array([ 0.08084438,  4.50819762, -3.58904201]),
 array([ 2.0270157 ,  0.12799001, -1.15500571]),
 array([-0.54668368, -0.1870132 ,  1.73369688])]